In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import torch
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import pickle
import gc

/home/richard/miniconda3/envs/esm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Get the absolute path of the current notebook
notebook_path = os.path.abspath('')

# Navigate to the project root (CS182-Final-Project)
project_root = os.path.dirname(notebook_path)
os.chdir(project_root)

# Now you can load the data using relative path from project root
print("Loading full dataset...")
full_data = pd.read_pickle('./data/benchmarkingGS_v1-0_similarityMeasure_sequence_v3-1.pkl')
display(full_data.head())


# Keep only the required columns
columns_to_keep = ['uniprotID_A', 'uniprotID_B', 'isInteraction', 'trainTest', 'sequence_A', 'sequence_B']
data = full_data[columns_to_keep]
display(data.head())

# Calculate dataset statistics
print("\n--- Full Dataset Statistics ---")

Loading full dataset...


,uniprotID_A,uniprotID_B,isInteraction,trainTest,RNAseqHPA,tissueHPA,tissueCellHPA,subcellularLocationHPA,bioProcessUniprot,cellCompUniprot,molFuncUniprot,domainUniprot,motifUniprot,Bgee,sequence_A,sequence_B
0,P28223,P41595,1,test2,0.160188,-0.449930,-0.060381,NaN,0.400892,0.404061,0.680414,0.0,0.790569,0.422078,MDILCEENTSLSSTTNSLMQLNDDTRLYSNDFNSGEANTSDAFNWT...,MALSYRVSELQSTIPEHILQSTFVHVISSNWSGLQTESIPEEMKQI...
1,O00161,P56962,1,train,0.825131,0.851690,0.675880,0.0,0.190693,0.200000,0.353553,0.0,0.000000,0.922975,MDNLSSEEIQQRAHQITDESLESTRRILGLAIESQDAGIKTITMLD...,MSEDEEKVKLRRLEPAIQKFIKIVIPTDLERLRKHQINIEKYQRCR...
2,P82979,Q01081,1,train,0.930790,0.954869,0.911887,0.0,0.547723,0.365148,0.408248,0.0,0.000000,NaN,MATETVELHKLKLAELKQECLARGLETKGIKQDLIHRLQAYLEEHA...,MAEYLASIFGTEKDKVNCSFYFKIGACRHGDRCSRLHNKPTFSQTI...
3,O60678,Q14524,1,train,0.219384,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,0.000000,NaN,MCSLASGATGGRGAVENEEDLPELSDSGDEAAWEDEDDADLPHGKQ...,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...
4,P10275,Q15648,1,train,0.500558,-0.311704,-0.371061,0.0,0.169811,0.375000,0.157895,0.0,0.000000,NaN,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,MKAQGETEESEKLSKMSSLLERLHAKFNQNRPWSETIKLVRQVMEK...


,uniprotID_A,uniprotID_B,isInteraction,trainTest,sequence_A,sequence_B
0,P28223,P41595,1,test2,MDILCEENTSLSSTTNSLMQLNDDTRLYSNDFNSGEANTSDAFNWT...,MALSYRVSELQSTIPEHILQSTFVHVISSNWSGLQTESIPEEMKQI...
1,O00161,P56962,1,train,MDNLSSEEIQQRAHQITDESLESTRRILGLAIESQDAGIKTITMLD...,MSEDEEKVKLRRLEPAIQKFIKIVIPTDLERLRKHQINIEKYQRCR...
2,P82979,Q01081,1,train,MATETVELHKLKLAELKQECLARGLETKGIKQDLIHRLQAYLEEHA...,MAEYLASIFGTEKDKVNCSFYFKIGACRHGDRCSRLHNKPTFSQTI...
3,O60678,Q14524,1,train,MCSLASGATGGRGAVENEEDLPELSDSGDEAAWEDEDDADLPHGKQ...,MANFLLPRGTSSFRRFTRESLAAIEKRMAEKQARGSTTLQESREGL...
4,P10275,Q15648,1,train,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,MKAQGETEESEKLSKMSSLLERLHAKFNQNRPWSETIKLVRQVMEK...



--- Full Dataset Statistics ---


In [3]:
def get_unique_proteins(df):
    unique_proteins = {}
    
    for _, row in df.iterrows():
        protein_A_id = row['uniprotID_A']
        protein_A_seq = row['sequence_A']
        if protein_A_id not in unique_proteins:
            unique_proteins[protein_A_id] = protein_A_seq
            
        protein_B_id = row['uniprotID_B']
        protein_B_seq = row['sequence_B']
        if protein_B_id not in unique_proteins:
            unique_proteins[protein_B_id] = protein_B_seq
            
    return unique_proteins


unique_proteins_dict = get_unique_proteins(data)

# Print how many unique proteins were found
print(f"Found {len(unique_proteins_dict)} unique proteins")

# Save to pickle file
os.makedirs('data/full_dataset', exist_ok=True)
with open('data/full_dataset/unique_proteins.pkl', 'wb') as f:
    pickle.dump(unique_proteins_dict, f)

print("Unique proteins saved to 'data/full_dataset/unique_proteins.pkl'")



Found 12026 unique proteins
Unique proteins saved to 'data/full_dataset/unique_proteins.pkl'


## Encode proteins with ESM C

In [4]:
from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig

device = "cuda" if torch.cuda.is_available() else "cpu"

if torch.cuda.is_available():
    target_gpu_index_for_fraction = 0

    print(f"Attempting to set memory fraction to 0.7 for GPU {target_gpu_index_for_fraction}")
    torch.cuda.set_per_process_memory_fraction(0.7, device=target_gpu_index_for_fraction)
    print(f"Call to set_per_process_memory_fraction for GPU {target_gpu_index_for_fraction} completed.")

model = ESMC.from_pretrained("esmc_300m").to(device)
print(f"[ESM-C] Loaded locally on {device}")
model_type = "local"

Attempting to set memory fraction to 0.7 for GPU 0
Call to set_per_process_memory_fraction for GPU 0 completed.


Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 101067.57it/s]


[ESM-C] Loaded locally on cuda


In [5]:
def get_protein_embedding(sequence):
    """
    Get protein embedding for a given sequence using the loaded ESM model.
    Optimized with torch.no_grad() for inference.
    """
    with torch.no_grad():  # Ensures no gradients are computed for model operations
        protein = ESMProtein(sequence=sequence)
        protein_tensor = model.encode(protein)  # Model inference step
        logits_output = model.logits(           # Model inference step
            protein_tensor,
            LogitsConfig(sequence=False, return_embeddings=True)
        )
        # Get the per-protein representation by mean-pooling across sequence length
        embedding = logits_output.embeddings
        # Remove the batch dimension (first dimension)
        embedding = embedding.squeeze(0)
    return embedding.cpu().to(dtype=torch.float16)


In [6]:
# Cell 2: Efficient protein embedding function
def embed_proteins_efficiently(proteins_dict, output_path='data/full_dataset/embeddings', batch_size=100):
    """
    Efficiently embeds proteins and saves with minimal storage requirements.
    
    Args:
        proteins_dict: Dictionary of protein IDs and sequences
        output_path: Directory to save embeddings
        batch_size: Number of proteins to process in each batch
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)
    
    # Track proteins processed
    total_proteins = len(proteins_dict)
    proteins_processed = 0
    
    # Process proteins in batches to manage memory
    protein_ids = list(proteins_dict.keys())
    
    # Set up storage stats tracking
    total_bytes = 0
    
    # Process in batches
    for i in range(0, total_proteins, batch_size):
        batch_ids = protein_ids[i:i+batch_size]
        batch_embeddings = {}
        
        # Process each protein in the batch
        for protein_id in tqdm(batch_ids, desc=f"Processing batch {i//batch_size + 1}/{(total_proteins-1)//batch_size + 1}"):
            sequence = proteins_dict[protein_id]
            
            # Get embedding using your function that returns float16 tensors
            embedding = get_protein_embedding(sequence)
            
            # Store in batch dictionary
            batch_embeddings[protein_id] = embedding
            proteins_processed += 1
        
        # Save this batch with compression
        batch_filename = os.path.join(output_path, f"embeddings_batch_{i//batch_size + 1}.pt")
        torch.save(batch_embeddings, batch_filename, _use_new_zipfile_serialization=True)
        
        # Get file size and add to total
        batch_size_bytes = os.path.getsize(batch_filename)
        total_bytes += batch_size_bytes
        
        print(f"Saved batch {i//batch_size + 1} ({len(batch_ids)} proteins): {batch_size_bytes/1024/1024:.2f} MB")
        
        # Free up memory
        del batch_embeddings
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
        
    # Save a manifest file with protein IDs and their batch locations
    manifest = {protein_id: f"embeddings_batch_{i//batch_size + 1}.pt" 
                for i, protein_id in enumerate(protein_ids)}
    
    with open(os.path.join(output_path, "embeddings_manifest.pkl"), "wb") as f:
        pickle.dump(manifest, f)
    
    print(f"\nEmbedding complete: {proteins_processed} proteins processed")
    print(f"Total storage used: {total_bytes/1024/1024/1024:.2f} GB")
    print(f"Average size per protein: {total_bytes/proteins_processed/1024:.2f} KB")

In [7]:
# Cell 3: Loading function
def load_protein_embedding(protein_id, embeddings_dir='data/full_dataset/embeddings'):
    """
    Efficiently load a single protein embedding when needed.
    """
    manifest_path = os.path.join(embeddings_dir, "embeddings_manifest.pkl")
    with open(manifest_path, "rb") as f:
        manifest = pickle.load(f)
    
    if protein_id not in manifest:
        raise KeyError(f"Protein ID {protein_id} not found in embeddings manifest")
    
    batch_file = manifest[protein_id]
    batch_path = os.path.join(embeddings_dir, batch_file)
    
    # Load only the specific batch file
    batch_data = torch.load(batch_path)
    
    # Return the specific embedding
    return batch_data[protein_id]

In [16]:
# Cell 4: Low memory implementation
def embed_proteins_low_memory(proteins_dict, output_path='data/full_dataset/embeddings', batch_size=10, 
                             start_batch=0, max_proteins_per_run=None):
    """
    Low-memory implementation to embed proteins and save them efficiently.
    Can resume from a specific batch and limits proteins processed in a single run.
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)
    
    # Track proteins processed
    total_proteins = len(proteins_dict)
    proteins_processed = 0
    
    # Process proteins in batches to manage memory
    protein_ids = list(proteins_dict.keys())
    
    # Load existing manifest if starting from a later batch
    manifest_path = os.path.join(output_path, "embeddings_manifest.pkl")
    if start_batch > 0 and os.path.exists(manifest_path):
        with open(manifest_path, "rb") as f:
            manifest = pickle.load(f)
    else:
        manifest = {}
    
    # Set up storage stats tracking
    total_bytes = 0
    
    # Apply limit if specified
    if max_proteins_per_run is not None:
        end_batch = min(start_batch + (max_proteins_per_run // batch_size) + 1, 
                       (total_proteins // batch_size) + 1)
    else:
        end_batch = (total_proteins // batch_size) + 1
    
    print(f"Will process from batch {start_batch+1} to {end_batch} (of {(total_proteins-1)//batch_size + 1} total)")
    
    # Process in batches
    total_batches = (total_proteins - 1) // batch_size + 1
    for batch_idx in range(start_batch, end_batch):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, total_proteins)
        batch_ids = protein_ids[start_idx:end_idx]
        
        if not batch_ids:
            continue
            
        batch_filename = os.path.join(output_path, f"embeddings_batch_{batch_idx + 1}.pt")
        
        # Skip if batch already exists (for resuming)
        if os.path.exists(batch_filename):
            print(f"Batch {batch_idx + 1} already exists, skipping...")
            # Update manifest entries for this batch
            for i, protein_id in enumerate(batch_ids):
                manifest[protein_id] = f"embeddings_batch_{batch_idx + 1}.pt"
            continue
            
        batch_embeddings = {}
        
        # Process each protein in the batch individually
        for protein_id in tqdm(batch_ids, desc=f"Processing batch {batch_idx + 1}/{total_batches}"):
            sequence = proteins_dict[protein_id]
        
        # Save this batch if we processed any proteins
        if batch_embeddings:
            torch.save(batch_embeddings, batch_filename, _use_new_zipfile_serialization=True)
            
            # Update manifest
            for pid in batch_embeddings.keys():
                manifest[pid] = f"embeddings_batch_{batch_idx + 1}.pt"
                
            # Get file size and add to total
            batch_size_bytes = os.path.getsize(batch_filename)
            total_bytes += batch_size_bytes
            
            print(f"Saved batch {batch_idx + 1} ({len(batch_embeddings)} proteins): {batch_size_bytes/1024/1024:.2f} MB")
        
        # Free up memory
        del batch_embeddings
        gc.collect()
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
        
        # Save manifest after each batch for safety
        with open(manifest_path, "wb") as f:
            pickle.dump(manifest, f)
    
    # Save final manifest file
    with open(manifest_path, "wb") as f:
        pickle.dump(manifest, f)
    
    print(f"\nEmbedding complete: {proteins_processed} proteins processed in this run")
    print(f"Total storage used in this run: {total_bytes/1024/1024/1024:.2f} GB")
    if proteins_processed > 0:
        print(f"Average size per protein: {total_bytes/proteins_processed/1024:.2f} KB")
    print(f"Total proteins in manifest: {len(manifest)}")

In [15]:
# Cell 5: Run embedding process
# Load unique proteins from the full dataset
with open('data/full_dataset/unique_proteins.pkl', 'rb') as f:
    unique_proteins = pickle.load(f)

# Set output directory
output_dir = 'data/full_dataset/embeddings'

# Run the embedding process
embed_proteins_low_memory(
    unique_proteins, 
    output_dir, 
    batch_size=100,
    start_batch=110,
    max_proteins_per_run=1000
)

Will process from batch 111 to 121 (of 121 total)


Processing batch 111/121:  10%|█         | 10/100 [00:02<00:19,  4.70it/s]

Truncating sequence for Q96MT7 from 1854 to 1500


Processing batch 111/121:  31%|███       | 31/100 [00:06<00:14,  4.67it/s]

Truncating sequence for P13942 from 1736 to 1500


...

Truncating sequence for Q5VT25 from 1732 to 1500


Processing batch 120/121: 100%|██████████| 100/100 [00:20<00:00,  4.90it/s]


Saved batch 120 (100 proteins): 101.04 MB


Processing batch 121/121: 100%|██████████| 26/26 [00:05<00:00,  5.04it/s]


Saved batch 121 (26 proteins): 24.61 MB

Embedding complete: 1026 proteins processed in this run
Total storage used in this run: 1.12 GB
Average size per protein: 1148.24 KB
Total proteins in manifest: 12026


In [17]:
# Cell 6: Merging functions
def merge_embeddings(embeddings_dir='data/full_dataset/embeddings', output_file=None):
    """
    Merge all protein embeddings from batch files into a single pickle file.
    
    Args:
        embeddings_dir: Directory containing the embedding batch files and manifest
        output_file: Path to save the merged embeddings (default: embeddings_merged.pkl in embeddings_dir)
    
    Returns:
        Path to the merged embeddings file
    """
    # Set default output file path if not provided
    if output_file is None:
        output_file = os.path.join(embeddings_dir, "embeddings_merged.pkl")
    
    # Load the manifest file
    manifest_path = os.path.join(embeddings_dir, "embeddings_manifest.pkl")
    with open(manifest_path, "rb") as f:
        manifest = pickle.load(f)
    
    print(f"Found {len(manifest)} proteins in manifest")
    
    # Get unique batch files
    batch_files = set(manifest.values())
    print(f"Found {len(batch_files)} batch files")
    
    # Create a dictionary to hold all embeddings
    all_embeddings = {}
    
    # Track file size for reporting
    total_loaded_bytes = 0
    
    # Process each batch file
    for batch_file in tqdm(batch_files, desc="Loading batch files"):
        batch_path = os.path.join(embeddings_dir, batch_file)
        
        # Get file size
        file_size = os.path.getsize(batch_path)
        total_loaded_bytes += file_size
        
        # Load batch embeddings
        batch_embeddings = torch.load(batch_path)
        
        # Add embeddings to master dictionary
        all_embeddings.update(batch_embeddings)
        
        # Clear memory
        del batch_embeddings
        gc.collect()
    
    print(f"Loaded {len(all_embeddings)} protein embeddings")
    print(f"Total loaded: {total_loaded_bytes/1024/1024/1024:.2f} GB")
    
    # Save the merged embeddings
    print(f"Saving merged embeddings to {output_file}...")
    with open(output_file, "wb") as f:
        pickle.dump(all_embeddings, f)
    
    # Get size of the merged file
    merged_size = os.path.getsize(output_file)
    print(f"Merged file size: {merged_size/1024/1024/1024:.2f} GB")
    
    return output_file

def merge_embeddings_compressed(embeddings_dir='data/full_dataset/embeddings', output_file=None):
    """
    Merge all protein embeddings from batch files into a single compressed pickle file.
    Uses highest compression level to minimize size.
    
    Args:
        embeddings_dir: Directory containing the embedding batch files and manifest
        output_file: Path to save the merged embeddings (default: embeddings_merged_compressed.pkl in embeddings_dir)
    
    Returns:
        Path to the merged embeddings file
    """
    import gzip
    
    # Set default output file path if not provided
    if output_file is None:
        output_file = os.path.join(embeddings_dir, "embeddings_merged_compressed.pkl")
    
    # Load the manifest file
    manifest_path = os.path.join(embeddings_dir, "embeddings_manifest.pkl")
    with open(manifest_path, "rb") as f:
        manifest = pickle.load(f)
    
    print(f"Found {len(manifest)} proteins in manifest")
    
    # Get unique batch files
    batch_files = set(manifest.values())
    print(f"Found {len(batch_files)} batch files")
    
    # Create a dictionary to hold all embeddings
    all_embeddings = {}
    
    # Track file size for reporting
    total_loaded_bytes = 0
    
    # Process each batch file
    for batch_file in tqdm(batch_files, desc="Loading batch files"):
        batch_path = os.path.join(embeddings_dir, batch_file)
        
        # Get file size
        file_size = os.path.getsize(batch_path)
        total_loaded_bytes += file_size
        
        # Load batch embeddings
        batch_embeddings = torch.load(batch_path)
        
        # Add embeddings to master dictionary
        all_embeddings.update(batch_embeddings)
        
        # Clear memory
        del batch_embeddings
        gc.collect()
    
    print(f"Loaded {len(all_embeddings)} protein embeddings")
    print(f"Total loaded: {total_loaded_bytes/1024/1024/1024:.2f} GB")
    
    # Save the merged embeddings with compression
    print(f"Saving compressed merged embeddings to {output_file}...")
    with gzip.open(output_file, "wb", compresslevel=9) as f:
        pickle.dump(all_embeddings, f)
    
    # Get size of the merged file
    merged_size = os.path.getsize(output_file)
    print(f"Merged compressed file size: {merged_size/1024/1024/1024:.2f} GB")
    print(f"Compression ratio: {merged_size/total_loaded_bytes:.2f}")
    
    return output_file

In [18]:
# Cell 7: Merge the embeddings
# Choose one of these options:

# Option 1: Basic merge - faster but larger file
merged_file = merge_embeddings()
print(f"Merged embeddings saved to: {merged_file}")

# Option 2: Merge with maximum compression - slower but smaller file
# merged_compressed_file = merge_embeddings_compressed()
# print(f"Compressed merged embeddings saved to: {merged_compressed_file}")

Found 12026 proteins in manifest
Found 121 batch files


Loading batch files: 100%|██████████| 121/121 [00:15<00:00,  7.70it/s]


Loaded 12026 protein embeddings
Total loaded: 11.23 GB
Saving merged embeddings to data/full_dataset/embeddings/embeddings_merged.pkl...
Merged file size: 11.23 GB
Merged embeddings saved to: data/full_dataset/embeddings/embeddings_merged.pkl


In [12]:
protein_embeddings = pickle.load(open('data/full_dataset/embeddings/embeddings_merged.pkl', 'rb'))


In [13]:
for key, value in protein_embeddings.items():
    print(f"Protein ID: {key}, Embedding shape: {value.shape}")

Protein ID: P28223, Embedding shape: torch.Size([473, 960])
Protein ID: P41595, Embedding shape: torch.Size([483, 960])
Protein ID: O00161, Embedding shape: torch.Size([213, 960])
Protein ID: P56962, Embedding shape: torch.Size([304, 960])
Protein ID: P82979, Embedding shape: torch.Size([212, 960])
Protein ID: Q01081, Embedding shape: torch.Size([242, 960])
Protein ID: O60678, Embedding shape: torch.Size([533, 960])
Protein ID: Q14524, Embedding shape: torch.Size([1502, 960])
Protein ID: P10275, Embedding shape: torch.Size([922, 960])
Protein ID: Q15648, Embedding shape: torch.Size([1502, 960])
Protein ID: P05413, Embedding shape: torch.Size([135, 960])
Protein ID: P05556, Embedding shape: torch.Size([800, 960])
Protein ID: Q99873, Embedding shape: torch.Size([373, 960])
Protein ID: O95573, Embedding shape: torch.Size([722, 960])
Protein ID: P08588, Embedding shape: torch.Size([479, 960])
Protein ID: P21917, Embedding shape: torch.Size([421, 960])
Protein ID: O75365, Embedding shape: t